In [1]:
%matplotlib inline
from matplotlib import pyplot as plt 

In [2]:
from glob import glob
import json
import csv
import os

import cv2

import numpy as np

In [7]:
import pickle as pkl

In [3]:
DATA_DIR = os.path.join(os.getcwd(), '../data')

In [5]:
RUNS_DIR = ['run1', 'run2']
RUNS_DATA = [os.path.join(DATA_DIR, run_dir) for run_dir in RUNS_DIR]

In [6]:
RUNS_DATA

['/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/run1',
 '/Users/Sumukh/Workspace/CarND/Term1/Projects/CarND-Behavioral-Cloning-P3/notebooks/../data/run2']

In [34]:
def read_image(img_path):
    return cv2.imread(img_path)

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def flip_image(img):
    return cv2.flip(img, 1)

In [35]:
def preprocess_image(img_path, gray=False, flip=False):
    img = read_image(img_path)
    assert img is not None
    img_flip = None
    if gray:
        img = grayscale(img)
    if flip:
        img_flip = flip_image(img)
    return img, img_flip

In [36]:
def preprocess(gray, flip):
    images = []
    angles = []
    for rdir in RUNS_DATA:
        log_file = os.path.join(rdir, 'driving_log.csv')
        with open(log_file, 'r') as _fh:
            log = csv.reader(_fh)
            for line in log:
                measurement = float(line[3])
                
                center, center_flip = preprocess_image(line[0], gray=gray, flip=flip)
                left, left_flip = preprocess_image(line[1], gray=gray, flip=flip)
                right, right_flip = preprocess_image(line[2], gray=gray, flip=flip)

                images.extend([center, left, right])
                angles.extend([measurement]*3)
                if flip:
                    images.extend([center_flip, left_flip, right_flip])
                    angles.extend([-measurement]*3)
    return np.array(images), np.array(angles)

In [37]:
flip = False
gray = False

images, angles = preprocess(gray, flip)

In [38]:
images.shape

(39915, 160, 320, 3)

In [40]:
angles.shape

(39915,)

In [41]:
images[34].shape

(160, 320, 3)

In [45]:
import h5py

In [74]:
f = h5py.File(os.path.join(DATA_DIR, 'driving_data.h5'), "w")
grp = f.create_group("data")
dset1 = grp.create_dataset("images", images.shape, 
                           dtype=np.float32)
dset2 = grp.create_dataset("angles", angles.shape,
                           dtype=np.float32)

In [75]:
dset1[...] = images

In [76]:
dset2[...] = angles

In [77]:
f.close()

In [87]:
flip = True
gray = True

images, angles = preprocess(gray, flip)

In [88]:
images.shape

(79830, 160, 320)

In [91]:
angles.shape

(79830,)

In [92]:
f = h5py.File(os.path.join(DATA_DIR, 'driving_data_gf.h5'), "w")
grp = f.create_group("data")
dset1 = grp.create_dataset("images", images.shape, 
                           dtype=np.float32)
dset2 = grp.create_dataset("angles", angles.shape,
                           dtype=np.float32)

In [93]:
dset1[...] = images

In [94]:
dset2[...] = angles

In [99]:
f.close()